<a href="https://colab.research.google.com/github/Amysoj-Louis/AI-Pdf-Question-Answering/blob/main/lichess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install zstandard
!pip install chess
import chess
import chess.pgn
import chess.svg
from IPython.display import SVG, display
import requests
from bs4 import BeautifulSoup
import zstandard as zstd
import os
from pathlib import Path

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.5 MB/s eta 0:00:00


In [9]:
url_to_test = 2
urls_with_standard = []

def fetch_urls():
    try:
        response = requests.get("https://database.lichess.org/")
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        for a_tag in soup.find_all("a"):
            href = a_tag.get('href')
            if href and "standard/" in href and "pgn.zst" in href and ".torrent" not in href:
                urls_with_standard.append(href.removeprefix("standard/"))

        urls_with_standard.sort()
        print(f"Found {len(urls_with_standard)} URLs")
    except requests.RequestException as e:
        print(f"Error fetching URLs: {e}")

def download_and_process_files():
    dctx = zstd.ZstdDecompressor()

    for i in range(min(url_to_test, len(urls_with_standard))):
        name = urls_with_standard[i]
        full_url = requests.compat.urljoin("https://database.lichess.org/standard/", name)

        try:
            # Download the .pgn.zst file
            response = requests.get(full_url)
            response.raise_for_status()  # Check for HTTP errors
            with open(name, 'wb') as file:
                file.write(response.content)
            # Decompress the .pgn.zst file
            input_file_path = Path(name)
            output_file_path = Path(name.replace(".zst",""))
            with input_file_path.open('rb') as compressed_file, output_file_path.open('wb') as output_file:
                dctx.copy_stream(compressed_file, output_file)

            # Delete the .pgn.zst file
            os.remove(input_file_path)
            print(f"Downloaded {name}")

        except requests.RequestException as e:
            print(f"Error downloading {name}: {e}")
        except OSError as e:
            print(f"File operation error for {name}: {e}")

    print("Downloading complete. Only .pgn files remain.")

if __name__ == "__main__":
    fetch_urls()
    download_and_process_files()


Found 139 URLs
Downloaded lichess_db_standard_rated_2013-01.pgn.zst
Downloaded lichess_db_standard_rated_2013-02.pgn.zst


In [ ]:
def count_games_in_pgn(pgn_file_path):
    try:
        game_count = 0
        with open(pgn_file_path, 'r') as pgn_file:
            while True:
                game = chess.pgn.read_game(pgn_file)
                if game is None:
                    break
                game_count += 1

                if game_count % 1000 == 0:
                    print(f"Processed {game_count} games...")

        return game_count
    except FileNotFoundError:
        print(f"Error: The file {pgn_file_path} does not exist.")
        return 0
    except Exception as e:
        print(f"Error: {e}")
        return 0

pgn_file_path = urls_with_standard[0]
num_games = count_games_in_pgn(pgn_file_path)
print(f"Number of games in the PGN file: {num_games}")


Processed 1000 games...
Processed 2000 games...
Processed 3000 games...
Processed 4000 games...
Processed 5000 games...
Processed 6000 games...
Processed 7000 games...
Processed 8000 games...
Processed 9000 games...
Processed 10000 games...
Processed 11000 games...
Processed 12000 games...
Processed 13000 games...
Processed 14000 games...
Processed 15000 games...
Processed 16000 games...
Processed 17000 games...
Processed 18000 games...
Processed 19000 games...
Processed 20000 games...
Processed 21000 games...
Processed 22000 games...
Processed 23000 games...
Processed 24000 games...
Processed 25000 games...
Processed 26000 games...
Processed 27000 games...
Processed 28000 games...
Processed 29000 games...
Processed 30000 games...
Processed 31000 games...
Processed 32000 games...
Processed 33000 games...
Processed 34000 games...
Processed 35000 games...
Processed 36000 games...
Processed 37000 games...
Processed 38000 games...
Processed 39000 games...
Processed 40000 games...
Processed

In [ ]:
with open("lichess_db_standard_rated_2013-01.pgn") as pgn:
    first_game = chess.pgn.read_game(pgn)

board = first_game.board()

display(SVG(chess.svg.board(board)))

move_count = 0

for move in first_game.mainline_moves():
    board.push(move)
    move_count += 1

    # Display the board after each move
    display(SVG(chess.svg.board(board)))

# Print total number of moves
print(f"Total number of moves: {move_count}")
